In [ ]:
# Install required packages
!pip install -q tensorflow keras opencv-python matplotlib gradio numpy pandas scikit-learn python-dotenv fpdf2

import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import matplotlib.pyplot as plt
import gradio as gr
import os
import random
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import tempfile
import datetime
import pandas as pd
from fpdf import FPDF

# Load your pre-trained model
model = keras.models.load_model('/content/drive/MyDrive/archive (1)/brain_tu.h5')

# Image preprocessing function
def preprocess_image(image, target_size=(224, 224)):
    if isinstance(image, str):
        img = cv2.imread(image)
    else:
        img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Prediction function
def predict_tumor(image):
    preprocessed_img = preprocess_image(image)
    prediction = model.predict(preprocessed_img)
    if prediction[0][0] > 0.5:
        return "Abnormal (Tumor Detected)", float(prediction[0][0])
    else:
        return "Normal (No Tumor)", float(1 - prediction[0][0])

# PDF Report Generation
def generate_pdf_report(image, result, confidence, email):
    temp_pdf = tempfile.NamedTemporaryFile(suffix=".pdf", delete=False)
    pdf_path = temp_pdf.name

    if isinstance(image, str):
        img_path = image
    else:
        img_path = "temp_image.jpg"
        cv2.imwrite(img_path, cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR))

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, "Brain Tumor Detection Report", 0, 1, 'C')
    pdf.ln(10)
    pdf.set_font("Arial", '', 12)
    pdf.cell(0, 10, f"Patient Email: {email}", 0, 1)
    pdf.cell(0, 10, f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", 0, 1)
    pdf.ln(10)
    pdf.image(img_path, x=50, w=110)
    pdf.ln(10)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "Diagnosis Results", 0, 1)
    pdf.set_font("Arial", '', 12)
    pdf.cell(0, 10, f"Result: {result}", 0, 1)
    pdf.cell(0, 10, f"Confidence: {confidence*100:.2f}%", 0, 1)
    pdf.ln(10)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, "Recommendations", 0, 1)
    pdf.set_font("Arial", '', 12)
    if "Abnormal" in result:
        pdf.multi_cell(0, 10, "Recommendation: Please consult with a neurosurgeon or oncologist for further evaluation and treatment options. Early detection and treatment can significantly improve outcomes.")
    else:
        pdf.multi_cell(0, 10, "Recommendation: No immediate action required. Regular checkups are recommended to monitor brain health.")
    pdf.ln(10)
    pdf.set_font("Arial", 'I', 10)
    pdf.cell(0, 10, "This report was generated by Brain Tumor Detection AI System", 0, 1, 'C')
    pdf.output(pdf_path)
    return pdf_path

# Email sending function
def send_email_with_pdf(receiver_email, pdf_path):
    sender_email = ""
    password = ""

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = "Your Brain Tumor Detection Report"

    body = """Please find attached your brain tumor detection report.
This report contains the analysis of your brain scan and recommendations based on our AI system's findings.
If you have any questions, please consult with a medical professional."""
    msg.attach(MIMEText(body, 'plain'))

    with open(pdf_path, "rb") as f:
        attach = MIMEApplication(f.read(), _subtype="pdf")
        attach.add_header('Content-Disposition', 'attachment', filename="brain_tumor_report.pdf")
        msg.attach(attach)

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        return True
    except Exception as e:
        print(f"Error sending email: {e}")
        return False

# Chatbot knowledge base
chatbot_knowledge = {
    "what is a brain tumor": "A brain tumor is a mass or growth of abnormal cells in the brain that can be benign (non-cancerous) or malignant (cancerous).",
    "symptoms": "Common symptoms include persistent headaches, seizures, nausea/vomiting, vision problems, balance issues, speech difficulties, and changes in personality or behavior.",
    "diagnosis": "Brain tumors are typically diagnosed using MRI or CT scans, neurological exams, and sometimes a biopsy where a sample of the tumor is examined.",
    "treatment": "Treatment options may include surgery to remove the tumor, radiation therapy, chemotherapy, targeted drug therapy, or a combination of these approaches.",
    "recovery": "Recovery depends on the tumor type, location, and treatment. Rehabilitation may include physical therapy, occupational therapy, and speech therapy.",
    "causes": "Most brain tumors have no known cause, though some risk factors include genetic conditions, family history, and exposure to radiation.",
    "types": "There are over 120 types of brain tumors. Common types include gliomas, meningiomas, pituitary adenomas, and medulloblastomas.",
    "survival rate": "Survival rates vary greatly depending on tumor type, grade, location, and patient factors. Some benign tumors have excellent prognosis.",
    "prevention": "There's no sure way to prevent brain tumors, but reducing exposure to radiation and environmental toxins may help lower risk.",
    "children": "Brain tumors are the most common solid tumors in children. Treatment approaches may differ from adults due to developing brains.",
}

# Chatbot function
def chatbot_response(message, chat_history):
    message_lower = message.lower()
    response = "I'm sorry, I don't have information about that. Please ask about brain tumor symptoms, diagnosis, treatment, or general information."
    for question in chatbot_knowledge:
        if question in message_lower:
            response = chatbot_knowledge[question]
            break
    chat_history.append((message, response))
    return "", chat_history

# Main processing function
def process_image_and_email(image, email):
    result, confidence = predict_tumor(image)
    pdf_path = generate_pdf_report(image, result, confidence, email)
    email_sent = send_email_with_pdf(email, pdf_path)
    email_status = "Email sent successfully!" if email_sent else "Failed to send email"
    fig = plt.figure(figsize=(8, 4))
    plt.text(0.1, 0.5, f"Result: {result}\nConfidence: {confidence*100:.2f}%\nStatus: {email_status}",
             fontsize=12, bbox=dict(facecolor='lightblue', alpha=0.5))
    plt.axis('off')
    return image, result, fig

# Custom UI CSS
css = """
body { background-color: #121212; color: #ffffff; font-family: 'Segoe UI', sans-serif; }
.gradio-container { background-color: #121212 !important; }
.header { text-align: center; color: #00FFAB; padding: 20px; }
.footer { text-align: center; font-size: 0.8em; color: #aaaaaa; margin-top: 20px; }
#upload-box { border: 2px dashed #00FFAB; border-radius: 15px; padding: 10px; }
button { background: linear-gradient(90deg, #00FFAB 0%, #00C2FF 100%) !important;
    border: none !important; border-radius: 10px !important; color: white !important; font-weight: bold !important; }
textarea, input { background-color: #1E1E1E !important; color: #ffffff !important; border: 1px solid #00FFAB !important; border-radius: 10px !important; }
"""

# Gradio UI
with gr.Blocks(title="Brain Tumor Detection AI", css=css) as demo:
    with gr.Column(elem_classes=["header"]):
        gr.Markdown("# 🧠 Brain Tumor Detection AI")
        gr.Markdown("Upload a brain scan image to detect potential tumors and receive a detailed report via email.")

    with gr.Tab("🧪 Tumor Detection"):
        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="📤 Upload Brain Scan", type="pil", elem_id="upload-box")
                email_input = gr.Textbox(label="📧 Your Email Address", placeholder="example@email.com")
                submit_btn = gr.Button("🚀 Analyze and Send Report")

            with gr.Column():
                image_output = gr.Image(label="🖼️ Processed Image", interactive=False)
                result_output = gr.Textbox(label="🧾 Detection Result")
                plot_output = gr.Plot(label="📊 Report Summary")

        submit_btn.click(
            fn=process_image_and_email,
            inputs=[image_input, email_input],
            outputs=[image_output, result_output, plot_output]
        )

    with gr.Tab("🤖 Brain Tumor Chatbot"):
        with gr.Row():
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(label="💡 Brain Tumor Information", bubble_full_width=False)
                msg = gr.Textbox(label="❓ Ask a question about brain tumors", placeholder="Type your question here...")
                clear = gr.Button("🧹 Clear Chat")
            with gr.Column(scale=1):
                gr.Markdown("### 🔥 Common Questions")
                gr.Examples(
                    examples=["What is a brain tumor?", "What are the symptoms?", "How is it diagnosed?", "What treatments are available?"],
                    inputs=msg
                )

        msg.submit(
            fn=chatbot_response,
            inputs=[msg, chatbot],
            outputs=[msg, chatbot]
        )
        clear.click(lambda: None, None, chatbot, queue=False)

    with gr.Column(elem_classes=["footer"]):
        gr.Markdown("### ⚙️ About this system")
        gr.Markdown("This AI system helps detect potential brain tumors from medical images. It provides a detailed report and includes an educational chatbot for brain tumor information.")

demo.launch(debug=True)


/tmp/ipython-input-4-972926458.py:193: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="💡 Brain Tumor Information", bubble_full_width=False)
/tmp/ipython-input-4-972926458.py:193: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(label="💡 Brain Tumor Information", bubble_full_width=False)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://335c8d1d7c6b32ba5b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step


/tmp/ipython-input-4-972926458.py:58: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", 'B', 16)
/tmp/ipython-input-4-972926458.py:59: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "Brain Tumor Detection Report", 0, 1, 'C')
/tmp/ipython-input-4-972926458.py:61: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", '', 12)
/tmp/ipython-input-4-972926458.py:62: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f"Patient Email: {email}", 0, 1)
/tmp/ipython-input-4-972926458.py:63: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.c